In [1]:
import os 
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
data = 'D:\EN_ARIS4_2\Proj_deep_learning\Data_Grayscale'

images = []
labels = []

folders = os.listdir(data)
folders

for folder in folders:
    folder_path = os.path.join(data, folder)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (100, 100))
        image = image.astype('float32') / 255.0

        images.append(image)
        labels.append(folder)

images = np.array(images)
images = np.expand_dims(images, axis=-1)

print(f"data value : {len(labels)}")
print("")

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
print(f"finish! label encoder")
print("")
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)
print(f"finish! train test split")
print("")

def build_thai_ocr_model():
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    # Output layer (assuming the number of Thai characters in the dataset, adjust as needed)
    model.add(Dense(len(folders), activation='softmax'))

    return model

model = build_thai_ocr_model()
model.summary()

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
print("finish! compile model")

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 32)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

In [ ]:
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('v4.h5')
print("Model saved as 'v4.h5'")

In [ ]:
# Function to predict Thai character from a new image
def predict_thai_character(image):
    # Preprocess the image (resize to 95x95, normalize)
    image = cv2.resize(image, (100, 100))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=(0, -1))

    # Predict the character
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)

    # Convert the integer back to the Thai character
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_label

# Test the prediction function with a new image
test_image = cv2.imread('captured_image.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

predicted_thai_character = predict_thai_character(test_image)
print(f'Predicted Thai character: {predicted_thai_character}')

In [ ]:
figure, axis = plt.subplots(3, 4)

i = 250

zero = cv2.imread(f'Data_Grayscale/0/0({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[0, 0].set_title(f'label : {pred}')
axis[0, 0].imshow(zero)
axis[0, 0].axis('off')

zero = cv2.imread(f'Data_Grayscale/1/1({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[0, 1].set_title(f'label : {pred}')
axis[0, 1].imshow(zero)
axis[0, 1].axis('off')

zero = cv2.imread(f'Data_Grayscale/2/2({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[0, 2].set_title(f'label : {pred}')
axis[0, 2].imshow(zero)
axis[0, 2].axis('off')

zero = cv2.imread(f'Data_Grayscale/3/3({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[0, 3].set_title(f'label : {pred}')
axis[0, 3].imshow(zero)
axis[0, 3].axis('off')

zero = cv2.imread(f'Data_Grayscale/4/4({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[1, 0].set_title(f'label : {pred}')
axis[1, 0].imshow(zero)
axis[1, 0].axis('off')

zero = cv2.imread(f'Data_Grayscale/5/5({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[1, 1].set_title(f'label : {pred}')
axis[1, 1].imshow(zero)
axis[1, 1].axis('off')

zero = cv2.imread(f'Data_Grayscale/6/6({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[1, 2].set_title(f'label : {pred}')
axis[1, 2].imshow(zero)
axis[1, 2].axis('off')

zero = cv2.imread(f'Data_Grayscale/7/7({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[1, 3].set_title(f'label : {pred}')
axis[1, 3].imshow(zero)
axis[1, 3].axis('off')

zero = cv2.imread(f'Data_Grayscale/8/8({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[2, 0].set_title(f'label : {pred}')
axis[2, 0].imshow(zero)
axis[2, 0].axis('off')

zero = cv2.imread(f'Data_Grayscale/9/9({i})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[2, 1].set_title(f'label : {pred}')
axis[2, 1].imshow(zero)
axis[2, 1].axis('off')

zero = cv2.imread(f'Data_Grayscale/0/0({260})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[2, 2].set_title(f'label : {pred}')
axis[2, 2].imshow(zero)
axis[2, 2].axis('off')

zero = cv2.imread(f'Data_Grayscale/5/5({260})_0.jpg', cv2.IMREAD_GRAYSCALE)
pred = predict_thai_character(zero)
axis[2, 3].set_title(f'label : {pred}')
axis[2, 3].imshow(zero)
axis[2, 3].axis('off')

plt.show()